In [10]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

In [11]:
AAD_TENANT_ID = "6babcaad-604b-40ac-a9d7-9fd97c0b779f"
KUSTO_URI = "https://llm-runners.eastus.kusto.windows.net"
KUSTO_INGEST_URI = "https://ingest-llm-runners.eastus.kusto.windows.net"
KUSTO_DATABASE = "llm-runner"

In [12]:
KCSB_DATA  = KustoConnectionStringBuilder.with_aad_user_token_authentication(KUSTO_URI,"..----")
KCSB_INGEST   = KustoConnectionStringBuilder.with_aad_user_token_authentication(KUSTO_INGEST_URI, "..-------")

In [13]:
KUSTO_CLIENT = KustoClient(KCSB_DATA)
CREATE_TABLE_COMMAND = ".create table InferencingEvents (PreciseTimeStamp: datetime, Region: string, ModelVersion: string, Provider: string, n_prompt: int, n_gen: int, stream: bool, finish_reason: string, ttft: decimal, mean_tbt: decimal, ttlt: decimal)"

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_TABLE_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,TableName,Schema,DatabaseName,Folder,DocString
0,InferencingEvents,"{""Name"":""InferencingEvents"",""OrderedColumns"":[...",llm-runner,None,None


In [14]:
CREATE_MAPPING_COMMAND = """.create table InferencingEvents ingestion csv mapping 'InferencingEvents_TSV_Mapping' '[{"Name":"PreciseTimeStamp","datatype":"datetime","Ordinal":0}, {"Name":"Region","datatype":"string","Ordinal":1}, {"Name":"ModelVersion","datatype":"string","Ordinal":2}, {"Name":"Provider","datatype":"string","Ordinal":3}, {"Name":"n_prompt","datatype":"int","Ordinal":4}, {"Name":"n_gen","datatype":"int","Ordinal":5}, {"Name":"stream","datatype":"bool","Ordinal":6}, {"Name":"finish_reason","datatype":"string","Ordinal":7}, {"Name":"ttft","datatype":"decimal","Ordinal":8}, {"Name":"mean_tbt","datatype":"decimal","Ordinal":9}, {"Name":"ttlt","datatype":"decimal","Ordinal":10}]'"""

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_MAPPING_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,Name,Kind,Mapping,LastUpdatedOn,Database,Table
0,InferencingEvents_TSV_Mapping,Csv,"[{""Name"":""PreciseTimeStamp"",""DataType"":""dateti...",2024-04-17 03:16:52.340720+00:00,llm-runner,InferencingEvents


In [35]:
import io
from datetime import datetime, timezone
p_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S.%f")
n_prompt= 123
n_gen=456
content = io.StringIO(f"{p_time},{n_prompt},{n_gen}")
    


In [20]:
from azure.kusto.data import DataFormat
from azure.kusto.ingest import QueuedIngestClient, IngestionProperties, FileDescriptor, BlobDescriptor, ReportLevel, ReportMethod

INGESTION_CLIENT = QueuedIngestClient(KCSB_INGEST)

INGESTION_PROPERTIES = IngestionProperties(database=KUSTO_DATABASE, table="TestEvents", data_format=DataFormat.CSV,
                                           ingestion_mapping_reference="TestEvents_CSV_Mapping", additional_properties={'ignoreFirstRecord': 'true'})

INGESTION_CLIENT.ingest_from_stream(content, ingestion_properties=INGESTION_PROPERTIES)

IngestionResult(status=IngestionStatus.QUEUED, database=llm-runner, table=TestEvents, source_id=ff17a293-a1d9-40f8-9c25-a4d39fa26f79, blob_uri=https://hdjkstrldllmrunners01.blob.core.windows.net/20240416-ingestdata-e5c334ee145d4b4-0/llm-runner__TestEvents__ff17a293-a1d9-40f8-9c25-a4d39fa26f79__stream.gz)

In [36]:
INGESTION_PROPERTIES = IngestionProperties(database=KUSTO_DATABASE, table="TestEvents", data_format=DataFormat.CSV,
                                           ingestion_mapping_reference="TestEvents_CSV_Mapping", additional_properties={'ignoreFirstRecord': 'false'})

INGESTION_CLIENT.ingest_from_stream(content, ingestion_properties=INGESTION_PROPERTIES)

IngestionResult(status=IngestionStatus.QUEUED, database=llm-runner, table=TestEvents, source_id=02035713-6f38-4b36-a725-5f68d8052ba6, blob_uri=https://pwrkstrldllmrunners00.blob.core.windows.net/20240416-ingestdata-e5c334ee145d4b4-0/llm-runner__TestEvents__02035713-6f38-4b36-a725-5f68d8052ba6__stream.gz)

In [22]:
with open("/tmp/hi.csv", 'r') as src:
    INGESTION_CLIENT.ingest_from_stream(src, ingestion_properties=INGESTION_PROPERTIES)

In [34]:
from datetime import datetime, timezone
p_time = "2024-03-14 00:00:00.0000000"
datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S.%f")

'2024-04-16 04:12:35.288579'

In [37]:
tenantid = "6babcaad-604b-40ac-a9d7-9fd97c0b779f"
clientid = "6e8d915a-460b-499c-a77f-10add633ef38"

In [ ]:
import adal
authority_url = 'https://login.microsoftonline.com/'+tenantid
context = adal.AuthenticationContext(authority_url)
token = context.acquire_token()

print(token["accessToken"])

In [4]:
import random
num = []
for _ in range(100):
    num.append(random.randint(0, 100))
    
print(sorted(num))

[0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 12, 13, 13, 13, 14, 15, 16, 16, 17, 19, 20, 20, 21, 21, 22, 22, 23, 24, 25, 25, 25, 25, 25, 28, 30, 30, 32, 35, 35, 35, 35, 36, 36, 37, 39, 40, 43, 46, 46, 48, 48, 49, 49, 49, 50, 51, 52, 53, 53, 54, 55, 57, 57, 61, 62, 62, 64, 64, 64, 64, 65, 65, 65, 65, 66, 69, 70, 74, 75, 76, 78, 81, 83, 83, 84, 85, 85, 87, 88, 91, 91, 94, 95, 97, 97, 98, 98, 100]


In [8]:
import random
min_prompt_len:int=50
max_prompt_len:int=5000
min_gen_len:int=10
max_gen_len:int=1000
i=10

while i>0:
    x = random.randint(min_prompt_len, max_prompt_len)
    j=10
    i-=1
    while j>0:
        y = random.randint(min_gen_len, max_gen_len)
        j-=1
        print(x,y)

3316 801
3316 56
3316 237
3316 845
3316 491
3316 506
3316 932
3316 347
3316 988
3316 896
3998 875
3998 950
3998 391
3998 359
3998 862
3998 229
3998 642
3998 965
3998 109
3998 259
2142 607
2142 76
2142 882
2142 96
2142 986
2142 373
2142 990
2142 437
2142 260
2142 751
2221 425
2221 439
2221 484
2221 489
2221 319
2221 281
2221 65
2221 956
2221 770
2221 362
3956 929
3956 201
3956 576
3956 673
3956 582
3956 794
3956 161
3956 399
3956 756
3956 499
4184 103
4184 188
4184 897
4184 859
4184 568
4184 295
4184 591
4184 231
4184 320
4184 551
2375 418
2375 291
2375 39
2375 61
2375 743
2375 99
2375 173
2375 161
2375 457
2375 289
620 810
620 866
620 866
620 866
620 11
620 954
620 286
620 605
620 479
620 962
4664 618
4664 796
4664 408
4664 585
4664 800
4664 95
4664 358
4664 168
4664 186
4664 414
2797 983
2797 714
2797 819
2797 818
2797 439
2797 930
2797 149
2797 353
2797 209
2797 25
